In [2]:
import numpy as np

from tqdm import tqdm_notebook as tqdm
from pommerman.constants import BOARD_SIZE

In [3]:
obs_path = "./dagger/train_data5/obs.npy"
done_path = "./dagger/train_data5/done.npy"
reward_path = "./dagger/train_data5/reward.npy"
labels_path = "./dagger/train_data5/labels.npy"

obs_new_path = './dagger/train_data5/obs.npy'
reward_new_path = './dagger/train_data5/reward.npy'
labels_new_path = './dagger/train_data5/labels.npy'
map_new_path = './dagger/train_data5/obs_map.npy'

In [4]:
obs = np.load(obs_path)
dones = np.load(done_path)
rewards = np.load(reward_path)
labels = np.load(labels_path)

In [6]:
print("done")

done


In [7]:
result_obs = []
result_rewards = []
result_labels = []

In [8]:
def featurize(obs):
    shape = (BOARD_SIZE, BOARD_SIZE, 1)

    def get_matrix(dict, key):
        res = dict[key]
        return res.reshape(shape).astype(np.float32)

    def get_map(board, item):
        map = np.zeros(shape)
        map[board == item] = 1
        return map

    board = get_matrix(obs, 'board')

    # TODO: probably not needed Passage = 0
    rigid_map = get_map(board, 1)               # Rigid = 1
    wood_map = get_map(board, 2)                # Wood = 2
    bomb_map = get_map(board, 3)                # Bomb = 3
    flames_map = get_map(board, 4)              # Flames = 4
    fog_map = get_map(board, 5)                 # TODO: not used for first two stages Fog = 5
    extra_bomb_map = get_map(board, 6)          # ExtraBomb = 6
    incr_range_map = get_map(board, 7)          # IncrRange = 7
    kick_map = get_map(board, 8)                # Kick = 8
    skull_map = get_map(board, 9)               # Skull = 9

    position = obs["position"]
    my_position = np.zeros(shape)
    my_position[position[0], position[1], 0] = 1

    team_mates = get_map(board, obs["teammate"].value) # TODO during documentation it should be an array

    enemies = np.zeros(shape)
    for enemy in obs["enemies"]:
        enemies[board == enemy.value] = 1

    bomb_blast_strength = get_matrix(obs, 'bomb_blast_strength')
    bomb_life = get_matrix(obs, 'bomb_life')

    ammo = np.full((BOARD_SIZE, BOARD_SIZE, 1), obs["ammo"])
    blast_strength = np.full((BOARD_SIZE, BOARD_SIZE, 1), obs["blast_strength"])
    can_kick = np.full((BOARD_SIZE, BOARD_SIZE, 1), int(obs["can_kick"]))

    obs = np.concatenate([my_position, enemies, team_mates, rigid_map,
                          wood_map, bomb_map, flames_map,
                          fog_map, extra_bomb_map, incr_range_map,
                          kick_map, skull_map, bomb_blast_strength,
                          bomb_life, ammo, blast_strength, can_kick], axis=2)
    return obs    

In [9]:
for i in range(4):
    ob = obs[:,i]
    reward = rewards[:,i]
    label = labels[:,i]
    
    idx = np.where(reward>-1)
    ob = ob[idx]
    reward = reward[idx]
    label = label[idx]
    
    result_obs.extend(ob)
    result_rewards.extend(reward)
    result_labels.extend(label)

In [10]:
result_obs = np.array(result_obs)
result_rewards = np.array(result_rewards)
result_labels = np.array(result_labels)

In [11]:
result_obs.shape

(1588939,)

In [12]:
np.save(obs_new_path, result_obs)
np.save(reward_new_path, result_rewards)
np.save(labels_new_path, result_labels)

In [13]:
temp = []
for obs in tqdm(result_obs):
    temp.append(featurize(obs))
result_obs = np.array(temp)

In [14]:
result_obs.shape

(1588939, 11, 11, 17)

In [15]:
np.sum(result_labels, axis=0) / np.sum(result_labels)

array([0.23930371, 0.189276  , 0.17057924, 0.18006733, 0.18439978,
       0.03637396], dtype=float32)

In [ ]:
np.save(map_new_path, result_obs)

In [ ]:
print("dat")